In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Home Credit 2024

# Import

In [ ]:
#import polars as pl  # a DataFrame library implemented in Rust language, designed to be blazingy fast and memory efficient
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

DATA_PATH = "/kaggle/input/home-credit-credit-risk-model-stability/csv_files/"

# Load Data

Base tables

In [ ]:
train_base = pd.read_csv(DATA_PATH + 'train/train_base.csv')
test_base = pd.read_csv(DATA_PATH + 'test/test_base.csv')

train_base.target = train_base.target.astype(np.int8)

In [ ]:
train_base.head()

Depth values:

* depth=0 - These are static features directly tied to a specific *case_id*.
* depth=1 - Each *case_id* has an associated historical record, indexed by *num_group1*.
* depth=2 - Each *case_id* has an associated historical record, indexed by both *num_group1* and *num_group2*.

### Depth 0

In [ ]:
train_static_0 = pd.concat([pd.read_csv(DATA_PATH + "train/train_static_0_0.csv"),
                            pd.read_csv(DATA_PATH + "train/train_static_0_1.csv")])
train_static_cb_0 = pd.read_csv(DATA_PATH + "train/train_static_cb_0.csv")
train_static_0 = train_static_0.merge(train_static_cb_0, on='case_id')


test_static_0 = pd.concat([pd.read_csv(DATA_PATH + "test/test_static_0_0.csv"),
                           pd.read_csv(DATA_PATH + "test/test_static_0_1.csv"),
                           pd.read_csv(DATA_PATH + "test/test_static_0_2.csv")])
test_static_cb_0 = pd.read_csv(DATA_PATH + "test/test_static_cb_0.csv")
test_static_0 = test_static_0.merge(test_static_cb_0, on='case_id')

In [ ]:
del train_static_cb_0
del test_static_cb_0

In [ ]:
train_static_0.head()

### Depth 1

In [ ]:
train_applprev_1 = pd.concat([pd.read_csv(DATA_PATH + "train/train_applprev_1_0.csv"),
                              pd.read_csv(DATA_PATH + "train/train_applprev_1_0.csv")])

#test_applprev_1 = pd.concat([pd.read_csv(DATA_PATH + "test/test_applprev_1_0.csv"),
#                             pd.read_csv(DATA_PATH + "test/test_applprev_1_1.csv"),
#                             pd.read_csv(DATA_PATH + "test/test_applprev_1_2.csv")])

In [ ]:
train_applprev_1.head()

In [ ]:
train_other_1 = pd.read_csv(DATA_PATH + "train/train_other_1.csv")

#test_other_1 = pd.read_csv(DATA_PATH + "test/test_other_1.csv")

In [ ]:
train_other_1.head()

In [ ]:
train_tax_registry_1 = pd.concat([pd.read_csv(DATA_PATH + "train/train_tax_registry_a_1.csv"),
                                  pd.read_csv(DATA_PATH + "train/train_tax_registry_b_1.csv"),
                                  pd.read_csv(DATA_PATH + "train/train_tax_registry_c_1.csv")])

#test_tax_registry_1 = pd.concat([pd.read_csv(DATA_PATH + "test/test_tax_registry_a_1.csv"),
#                                 pd.read_csv(DATA_PATH + "test/test_tax_registry_b_1.csv"),
#                                 pd.read_csv(DATA_PATH + "test/test_tax_registry_c_1.csv")])

In [ ]:
train_tax_registry_1.head()

In [ ]:
train_credit_bureau_1 = pd.concat([pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_1_0.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_1_1.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_1_2.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_1_3.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_b_1.csv")])

#test_credit_bureau_1 = pd.concat([pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_1_0.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_1_1.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_1_2.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_1_3.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_b_1.csv")])

In [ ]:
train_credit_bureau_1.head()

In [ ]:
train_deposit_1 = pd.read_csv(DATA_PATH + "train/train_deposit_1.csv")

#test_deposit_1 = pd.read_csv(DATA_PATH + "test/test_deposit_1.csv")

In [ ]:
train_deposit_1.head()

In [ ]:
train_person_1 = pd.read_csv(DATA_PATH + "train/train_person_1.csv")

test_person_1 = pd.read_csv(DATA_PATH + "test/test_person_1.csv")

In [ ]:
train_person_1.head()

In [ ]:
train_debitcard_1 = pd.read_csv(DATA_PATH + "train/train_debitcard_1.csv")

#test_debitcard_1 = pd.read_csv(DATA_PATH + "test/test_debitcard_1.csv")

In [ ]:
train_debitcard_1.head()

### Depth 2

In [ ]:
train_applprev_2 = pd.read_csv(DATA_PATH + "train/train_applprev_2.csv")

#test_applprev_2 = pd.read_csv(DATA_PATH + "test/test_applprev_2.csv")

In [ ]:
train_person_2 = pd.read_csv(DATA_PATH + "train/train_person_2.csv")

#test_person_2 = pd.read_csv(DATA_PATH + "test/test_person_2.csv")

In [ ]:
train_credit_bureau_2 = pd.concat([pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_0.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_1.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_2.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_3.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_4.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_5.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_6.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_7.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_8.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_9.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_a_2_10.csv"),
                                   pd.read_csv(DATA_PATH + "train/train_credit_bureau_b_2.csv")])

#test_credit_bureau_2 = pd.concat([pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_0.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_1.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_2.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_3.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_4.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_5.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_6.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_7.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_8.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_9.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_10.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_a_2_11.csv"),
#                                  pd.read_csv(DATA_PATH + "test/test_credit_bureau_b_2.csv")])

# Exploration

### Base Data

In [ ]:
train_base.info()

In [ ]:
train_base.describe()

In [ ]:
ax = sns.countplot(x=train_base['target'])
plt.bar_label(container=ax.containers[0], labels=[f"{val:.2f}%" for val in train_base['target'].value_counts(normalize=True).values*100])
plt.title('Target distribution.')
plt.show()

### Depth 0 Data

In [ ]:
train_static_0.head()

In [ ]:
plt.figure(figsize=(10,50))
sns.barplot(x=train_static_0.isna().sum(), y=train_static_0.columns.to_list(), order=train_static_0.isna().sum().sort_values().index, orient='h')
#plt.xticks(rotation=90)
plt.axvline(x=train_static_0.shape[0], c='red')
plt.show()

### Depth 1 Data

In [ ]:
# 1
train_applprev_1
train_other_1
train_tax_registry_1
train_credit_bureau_1
train_deposit_1
train_person_1
train_debitcard_1

In [ ]:
train_person_1.info()

In [ ]:
train_person_1.head()

# Preprocessing

In [ ]:
# We need to use aggregation functions in tables with depth > 1, so tables that contain num_group1 column or also num_group2 column.
train_person_1_feats_1 = (
    train_person_1.groupby("case_id")
    .agg(
        mainoccupationinc_384A_max=("mainoccupationinc_384A", "max"),
        mainoccupationinc_384A_any_selfemployed=("incometype_1044T", lambda x: (x == "SELFEMPLOYED").max())
    )
    .reset_index()
)

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = (
    train_person_1.loc[train_person_1["num_group1"] == 0, ["case_id", "num_group1", "housetype_905L"]]
    .drop("num_group1", axis=1)
    .rename(columns={"housetype_905L": "person_housetype"})
)


In [ ]:
# We will process in this examples only A-type and M-type columns, so we need to select them.
selected_static_cols = []
for col in train_static_0.columns:
    if col[-1] in ("A", "M"):
        selected_static_cols.append(col)
print(selected_static_cols)

In [ ]:
# Join all tables together.
data = (
    train_base
    .merge(train_static_0[["case_id"] + selected_static_cols], how="left", on="case_id")
    .merge(train_person_1_feats_1, how="left", on="case_id")
    .merge(train_person_1_feats_2, how="left", on="case_id")
)


In [ ]:
data.head()

In [ ]:
test_person_1_feats_1 = (
    test_person_1.groupby("case_id")
    .agg(
        mainoccupationinc_384A_max=("mainoccupationinc_384A", "max"),
        mainoccupationinc_384A_any_selfemployed=("incometype_1044T", lambda x: (x == "SELFEMPLOYED").max())
    )
    .reset_index()
)

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
test_person_1_feats_2 = (
    test_person_1.loc[test_person_1["num_group1"] == 0, ["case_id", "num_group1", "housetype_905L"]]
    .drop("num_group1", axis=1)
    .rename(columns={"housetype_905L": "person_housetype"})
)


data_submission = (
    test_base
    .merge(test_static_0[["case_id"]+selected_static_cols], how="left", on="case_id")
    .merge(test_person_1_feats_1, how="left", on="case_id")
    .merge(test_person_1_feats_2, how="left", on="case_id")
)

In [ ]:
data_submission.shape

# Feature Engineering

In [ ]:
def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

In [ ]:
case_ids = data["case_id"].unique()
np.random.shuffle(case_ids)
case_ids_train, case_ids_valid = train_test_split(case_ids, train_size=0.8, random_state=1)

cols_pred = []
for col in data.columns:
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)

print(cols_pred)

X_train, y_train = (
    data[data['case_id'].isin(case_ids_train)][cols_pred].reset_index(drop=True),
    data[data['case_id'].isin(case_ids_train)][['case_id', 'target']].reset_index(drop=True)
)

X_valid, y_valid = (
    data[data['case_id'].isin(case_ids_valid)][cols_pred].reset_index(drop=True),
    data[data['case_id'].isin(case_ids_valid)][['case_id', 'target']].reset_index(drop=True)
)
for df in [X_train, X_valid]:
    df = convert_strings(df)

In [ ]:
X_test = data_submission[cols_pred].copy()
X_test = convert_strings(X_test)
categorical_cols = X_train.select_dtypes(include=['category']).columns

for col in categorical_cols:
    train_categories = set(X_train[col].cat.categories)
    submission_categories = set(X_test[col].cat.categories)
    new_categories = submission_categories - train_categories

    # Replace new categories with 'Unknown'
    X_test.loc[X_test[col].isin(new_categories), col] = "Unknown"

    # Cast to the appropriate dtype
    new_dtype = pd.CategoricalDtype(categories=train_categories, ordered=True)
    X_train[col] = X_train[col].astype(new_dtype)
    X_test[col] = X_test[col].astype(new_dtype)


In [ ]:
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")

# Modeling

In [ ]:
# TBD

# Results

In [ ]:
# TBD

# Submit

In [ ]:
# TBD